In [1]:
import numpy as np
import pandas as pd
from stopsignalmetrics import StopData, SSRTmodel, PostStopSlow, Violations, StopSummary


import matplotlib.pyplot as plt
import seaborn as sns

# MTURK DATA

In [2]:
variable_dict = {
   "columns": {
      "ID": "worker_id",
      "block": "current_block",
      "condition": "SS_trial_type",
      "SSD": "SS_delay",
      "goRT": "rt",
      "stopRT": "rt",
      "response": "key_press",
      "correct_response": "correct_response",
      "choice_accuracy": "choice_accuracy"
   },
   "key_codes": {
      "go": "go",
      "stop": "stop",
      "correct": 1,
      "incorrect": 0,
      "noResponse": -1
   }
}
stop_data = StopData(var_dict=variable_dict)

# 1 Subject

In [3]:
subj_file = '../stopsignalmetrics/data/example_mturk_individual.csv'
subj_df = pd.read_csv(subj_file)
subj_df = subj_df.fillna(-1.)
cleaned_subj = stop_data.fit_transform(subj_df)

In [4]:
ssrt_model = SSRTmodel(model='all')
metrics = ssrt_model.fit_transform(cleaned_subj)
metrics

{'SSRT': {'mean': 301.53228449688623,
  'integration': 279.6296296296296,
  'omission': 282.6296296296296,
  'replacement': 282.6296296296296},
 'mean_SSD': 295.3703703703704,
 'p_respond': 0.5,
 'max_RT': 1675.0,
 'mean_go_RT': 596.9026548672566,
 'mean_stopfail_RT': 520.8888888888889,
 'omission_count': 1,
 'omission_rate': 0.008771929824561403,
 'go_acc': 0.911504424778761,
 'stopfail_acc': 0.0}

# Group 1

In [10]:
group_file = '../stopsignalmetrics/data/example_mturk_group_minimal.csv'
group_df = pd.read_csv(group_file)
# group_df = group_df.reset_index()

cleaned_group = stop_data.fit_transform(group_df)
cleaned_group

AssertionError: missing SS_trial_type from raw data df columns

In [11]:
group_df

,worker_id,current_block,stop_signal_condition,SS_delay,correct_response,key_press,correct_trial,rt
0,s264,0,go,250,90,90,1,622
1,s264,0,go,250,77,77,1,519
2,s264,0,go,250,77,77,1,491
3,s264,0,go,250,77,77,1,389
4,s264,0,go,250,77,77,1,314
...,...,...,...,...,...,...,...,...
476,s142,1,go,300,90,90,1,532
477,s142,1,go,300,77,77,1,469
478,s142,1,stop,300,-1,-1,1,-1
479,s142,1,stop,350,-1,90,0,382


In [ ]:
ind_test = Violations().fit_transform(cleaned_group, level='group')

In [ ]:
ind_test

In [ ]:
all_ssdvals = g_test.index.levels[1]

In [ ]:
all_ssdvals

In [ ]:
g_test.loc[(slice(None) , 300.0), ]

In [ ]:
ssrt_model = SSRTmodel()
cleaned_group.groupby('ID').apply(ssrt_model.fit_transform).apply(pd.Series)

In [ ]:
StopSummary().fit_transform(cleaned_group, level='group')

In [ ]:
Violations().fit_transform(cleaned_group, level='group')

In [ ]:
ind_test = ind_test.set_index('SSD')

# Group 2 - Recreating Figure 1b

In [ ]:
# load in data
group2_file = 'example_data/DataFixedSSDs2.xlsx'
group2_df = pd.read_excel(group2_file)
group2_df = group2_df.replace(r'^\s*$', np.nan, regex=True).replace('?', np.nan)

#build up "correct" responses for stop trials
addon_file = 'example_data/FixedSSD2StopTrialChoiceAccuracyInput.xlsx'
addon_df = pd.read_excel(addon_file)

group2_df['StopTrialCorrectResponse'] = np.nan

# circle response
for shape in ['Circle', 'Rhombus', 'Square', 'Triangle']:
    group2_df.loc[(group2_df['TrialType']=='stop') & (addon_df['Unnamed: 5']==f'{shape.lower()}.bmp'), 'StopTrialCorrectResponse'] = \
    addon_df.loc[(group2_df['TrialType']=='stop') & (addon_df['Unnamed: 5']==f'{shape.lower()}.bmp'), f'{shape}Response']

#combine go and stop into single columns
group2_df['response'] = np.where(group2_df['GoTrialResponse'].isnull(), group2_df['StopTrialResponse'], group2_df['GoTrialResponse'])
group2_df['CorrectResponse'] = np.where(group2_df['GoTrialCorrectResponse'].isnull(), group2_df['StopTrialCorrectResponse'], group2_df['GoTrialCorrectResponse'])
group2_df['CorrectResponse'] = group2_df['CorrectResponse'].str.lower()

# Preprocess data

In [ ]:
variable_dict = {
   "columns": {
      "ID": "Subject",
      "block": "Block",
      "condition": "TrialType",
      "SSD": "StopSignalDelay",
      "goRT": "GoRT",
      "stopRT": "StopFailureRT",
      "response": "response",
      "correct_response": "CorrectResponse",
      "choice_accuracy": "choice_accuracy"
   },
   "key_codes": {
      "go": "go",
      "stop": "stop",
      "correct": 1,
      "incorrect": 0
   }
}

stop_data = StopData(var_dict=variable_dict)
cleaned_df = stop_data.fit_transform(group2_df)

# Get Group Violations - 1 line!

In [ ]:
va_df = Violations().fit_transform(cleaned_df, level='group')

# Make a pretty & compelling plot

In [ ]:
# Pivot & Plot
pivot_df = va_df.pivot_table(values='mean_violation', index=['SSD'],
                        columns=['ID'])


fig = plt.figure(figsize=(10,8))
plt.plot(pivot_df, linewidth=0, color='c', marker='o', alpha=.3, ) #plot individuals
ax = sns.lineplot(x='SSD', y="mean_violation", data=va_df, color='c')
ax.axis([0,500,-300,250])
ax.plot([0,pivot_df.index.max()],[0,0],color='k',linestyle=':',linewidth=3)

xticks=np.arange(0,pivot_df.index.max()+50,50)
xticks = [int(i) for i in xticks]
if len(xticks)> 16:
    xticks = [i for i in xticks if i%100==0]
ax.set_xticks(xticks)
ax.set_xticklabels(xticks,fontsize=18) 
yticks=np.arange(-300,250+50,50)
ax.set_yticks(yticks)
ax.set_yticklabels(yticks,fontsize=18)

ax.set_xlabel('Stop-signal delay (ms)',fontsize=24)
_ = ax.set_ylabel('Stop failure RT - No-stop RT',fontsize=24) 
plt.show()
plt.close()

In [ ]:
import json

In [ ]:
with open("sample.json", "w") as outfile:  
    json.dump({'check': np.nan}, outfile)

